In [1]:
from ProcessingEmbeddings import *
from HardDebias import *
from DoubleHardDebias import *
from INLP import *
from utils import *

In [2]:
import codecs
import json
import numpy as np
import sklearn
from sklearn.svm import LinearSVC, SVC


In [3]:
glove=Embeddings('glove-wiki-gigaword-50')

Loading glove-wiki-gigaword-50 embeddings
vectors shape: (400000, 50), word2idx length: 400000, vocab length: 400000


In [4]:
vectors=glove.vectors
word2idx=glove.word2idx
vocab=glove.words
dict_vectors = glove.get_word_vector_dict()

In [5]:
np.isnan(vectors).any()

False

In [6]:
#Gender specific vocabulary:
gender_specific=[]
with open('./Data/male_word_file.txt') as f:
    gender_specific = [line.strip() for line in f]

with open('./Data/female_word_file.txt') as f:
    for l in f:
        gender_specific.append(l.strip())

with codecs.open('./Data/gender_specific_full.json') as f:
    gender_specific.extend(json.load(f))

In [7]:
# Getting a limited vocabulary to debias the embeddings.
vocab_cleaned, vectors_cleaned, word2idx_cleaned, dict_vec_cleaned = glove.limit_vocab(vectors, word2idx, vocab, exclude=gender_specific)

100%|██████████| 400000/400000 [00:00<00:00, 631139.53it/s]


Size of limited vocabulary: 326614


In [18]:
#Getting the definitional sets to calculate afterwards the gender direction. The first 10 gender sets were proposed by Bolukbasi et al. (2016)
#Definitional sets for race where proposed by Manzini et al. in Multiclass debiasing of embeddings: https://github.com/TManzini/DebiasMulticlassWordEmbedding/blob/master/Debiasing/data/vocab/race_attributes_optm.json

def_sets={
    "gender" : [
    ['she', 'he'], ['herself', 'himself'], ['her', 'his'], ['daughter', 'son'], ['girl', 'boy'],
    ['mother', 'father'], ['woman', 'man'], ['mary', 'john'], ['gal', 'guy'], ['female', 'male'],['aunt', 'uncle']],
    
    "race":[
		["black", "caucasian", "asian"],
		["african", "caucasian", "asian"],
		["black", "white", "asian"],
		["africa", "america", "asia"],
		["africa", "america", "china"],
		["africa", "europe", "asia"]
    ], 
    "religion":[
		["judaism", "christianity", "islam"],
		["jew", "christian", "muslim"],
    ["synagogue", "church", "mosque"],
    ["torah", "bible", "quran"],
    ["rabbi", "priest", "imam"]
	]}

In [ ]:
equalizing_lists = {
    "gender": [
        ["monastery", "convent"], ["spokesman", "spokeswoman"], ["Catholic_priest", "nun"], ["Dad", "Mom"], ["Men", "Women"],
        ["councilman", "councilwoman"], ["grandpa", "grandma"], ["grandsons", "granddaughters"], ["prostate_cancer", "ovarian_cancer"],
        ["testosterone", "estrogen"], ["uncle", "aunt"], ["wives", "husbands"], ["Father", "Mother"], ["Grandpa", "Grandma"],
        ["He", "She"], ["boy", "girl"], ["boys", "girls"], ["brother", "sister"], ["brothers", "sisters"], ["businessman", "businesswoman"],
        ["chairman", "chairwoman"], ["colt", "filly"], ["congressman", "congresswoman"], ["dad", "mom"], ["dads", "moms"], ["dudes", "gals"],
        ["ex_girlfriend", "ex_boyfriend"], ["father", "mother"], ["fatherhood", "motherhood"], ["fathers", "mothers"], ["fella", "granny"],
        ["fraternity", "sorority"], ["gelding", "mare"], ["gentleman", "lady"], ["gentlemen", "ladies"], ["grandfather", "grandmother"],
        ["grandson", "granddaughter"], ["he", "she"], ["himself", "herself"], ["his", "her"], ["king", "queen"], ["kings", "queens"],
        ["male", "female"], ["males", "females"], ["man", "woman"], ["men", "women"], ["nephew", "niece"], ["prince", "princess"], 
        ["schoolboy", "schoolgirl"], ["son", "daughter"], ["sons", "daughters"], ["twin_brother", "twin_sister"]],

    "race": [
        ["black", "caucasian", "asian"],
      		["african", "caucasian", "asian"],
      		["black", "white", "asian"],
      		["africa", "america", "asia"],
      		["africa", "america", "china"],
      		["africa", "europe", "asia"]
    ],
    "def_sets_religion": [
        ["judaism", "christianity", "islam"],
      		["jew", "christian", "muslim"],
        ["synagogue", "church", "mosque"],
        ["torah", "bible", "quran"],
        ["rabbi", "priest", "imam"]
    ]}


In [9]:
#Equalizing pairs were first published by Bolukbasi et al. in https://github.com/tolga-b/debiaswe/blob/master/data/equalize_pairs.json
equalizing_list=[["monastery", "convent"], ["spokesman", "spokeswoman"], ["Catholic_priest", "nun"], ["Dad", "Mom"], ["Men", "Women"], ["councilman", "councilwoman"], ["grandpa", "grandma"], ["grandsons", "granddaughters"], ["prostate_cancer", "ovarian_cancer"], ["testosterone", "estrogen"], ["uncle", "aunt"], ["wives", "husbands"], ["Father", "Mother"], ["Grandpa", "Grandma"], ["He", "She"], ["boy", "girl"], ["boys", "girls"], ["brother", "sister"], ["brothers", "sisters"], ["businessman", "businesswoman"], ["chairman", "chairwoman"], ["colt", "filly"], ["congressman", "congresswoman"], ["dad", "mom"], ["dads", "moms"], ["dudes", "gals"], ["ex_girlfriend", "ex_boyfriend"], ["father", "mother"], ["fatherhood", "motherhood"], ["fathers", "mothers"], ["fella", "granny"], ["fraternity", "sorority"], ["gelding", "mare"], ["gentleman", "lady"], ["gentlemen", "ladies"], ["grandfather", "grandmother"], ["grandson", "granddaughter"], ["he", "she"], ["himself", "herself"], ["his", "her"], ["king", "queen"], ["kings", "queens"], ["male", "female"], ["males", "females"], ["man", "woman"], ["men", "women"], ["nephew", "niece"], ["prince", "princess"], ["schoolboy", "schoolgirl"], ["son", "daughter"], ["sons", "daughters"], ["twin_brother", "twin_sister"]]

### Hard-Debias Algorithm

#### Gender

In [10]:
# To compute the gender bias, we need to get the embeddings of "he" and "she"
he_embed = dict_vectors['he']
she_embed = dict_vectors['she']

# Using the gender bias function to compute the bias of all the words in the limited dataset
#We create a dictionary with the word as key and the bias as value
gender_bias_original = utils.compute_bias(dict_vec_cleaned, he_embed, she_embed)

In [11]:
def prepare_def_sets_subspace(list_def_sets):
  def_sets={i: v for i, v in enumerate(list_def_sets)}
  return def_sets

In [19]:
def_set_gender=utils.prepare_def_sets_subspace(def_sets["gender"])
def_set_race=utils.prepare_def_sets_subspace(def_sets["race"])
def_set_religion=utils.prepare_def_sets_subspace(def_sets["religion"])

In [13]:
import HardDebias as hd

In [16]:
debiased_vectors, debiased_vocab, debiased_word2idx= hard_debias(vectors,
                             dict_vec_cleaned, 
                             word2idx_cleaned,
                             vocab_cleaned, 
                             equalizing_list, 
                             def_set_gender,
                             1
                             )

number of words considered: 2
Running PCA with 1 components


###  Double-Hard Debias

In [15]:
biased_w2i, biased_vocab, female_words, male_words, y_true=utils.getting_biased_words(gender_bias_original, def_sets["def_sets_gender"], 1000,word2idx_cleaned)

In [16]:
np.isnan(vectors).any()

False

In [17]:
precisions, optimal_direction = getting_optimal_direction(vectors, word2idx, biased_w2i, biased_vocab, male_words, female_words, y_true,def_sets["def_sets_gender"])

Component: 0, 

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [19]:
dh_debiased_vectors= double_hard_debias(vectors, word2idx, word2idx, vocab, [optimal_direction], def_sets["def_sets_gender"])

NameError: name 'optimal_direction' is not defined

In [20]:
dh_debiased_vocab_limited, dh_debiased_vectors_limited, dh_debiased_word2idx_limited = glove.limit_vocab(dh_debiased_vectors, word2idx, vocab, exclude = gender_specific)

NameError: name 'dh_debiased_vectors' is not defined

### INLP

In [16]:
num_vectors_per_class = 7500

#gender_direction=algorithm1.find_gender_direction(vectors, word2idx_cleaned)

#gender_vecs = [glove.model[p[0]] - glove.model[p[1]] for p in gendered_pairs]
#pca = PCA(n_components=1)
#pca.fit(gender_vecs)
gender_direction = identify_bias_subspace(dict_vec_cleaned, def_set_gender, 1, centralizing=True)
gender_direction = np.squeeze(gender_direction)

gender_unit_vec = gender_direction/np.linalg.norm(gender_direction)
masc_words_and_scores, fem_words_and_scores, neut_words_and_scores = getting_classes_for_INLP(gender_vector=gender_direction, model=glove.model, n = 7500)

masc_words, masc_scores = list(zip(*masc_words_and_scores))
neut_words, neut_scores = list(zip(*neut_words_and_scores))
fem_words, fem_scores = list(zip(*fem_words_and_scores))
masc_vecs, fem_vecs = glove.get_vectors_from_list(masc_words), glove.get_vectors_from_list(fem_words)
neut_vecs = glove.get_vectors_from_list(neut_words)

n = min(3000, num_vectors_per_class)
all_significantly_biased_words = masc_words[:n] + fem_words[:n]
all_significantly_biased_vecs =  np.concatenate((masc_vecs[:n], fem_vecs[:n]))
all_significantly_biased_labels = np.concatenate((np.ones(n, dtype = int),
                                                  np.zeros(n, dtype = int)))

all_significantly_biased_words, all_significantly_biased_vecs, all_significantly_biased_labels = sklearn.utils.shuffle(
all_significantly_biased_words, all_significantly_biased_vecs, all_significantly_biased_labels)
#print(np.random.choice(masc_words, size = 75))
print("TOP MASC:",masc_words[:50] )
print("-------------------------")
print("TOP FEM:", fem_words[:50])
print("-------------------------")
print("Random Neutral:", neut_words[:50])

number of words considered: 2
Running PCA with 1 components
TOP MASC: ('vulva', 'ilirska', 'sopra', 'seacole', 'konstantinovna', 'phagan', 'ivanovna', 'chromatids', 'sistership', 'lactating', 'aldermanbury', 'chromatid', 'antipolis', 'masumeh', 'ruwart', 'benedicta', 'agia', 'dasi', 'triada', 'maharani', 'gothel', 'woolnoth', 'miscarries', 'popova', 'mizzenmast', 'catharina', 'czestochowa', 'nikolayevna', 'lehzen', 'desnuda', 'herøy', 'murten', 'caesonia', 'mulleavy', 'aghia', 'ludwika', 'ignacia', 'parvomay', 'ferroviario', 'gaitskill', 'tanztheater', 'skłodowska-curie', 'vissi', 'jumonji', 'philomela', 'mahatmya', 'grebenkina', 'rafaela', 'viletta', 'romanova')
-------------------------
TOP FEM: ('inienger', 'boonyaratkalin', 'abdulsalami', 'jets', 'sonthi', 'scuds', 'voreqe', 'technicals', 'strategists', 'fired', 'valiquette', 'ac-130', 'tactic', 'patriot', 'defensive', 'singirok', 'kassam', 'greifeld', 'offensive', 'boonyaratglin', 'staubach', 'tactics', 'dannatt', 'sacking', 'kfir

In [17]:
bias_direction = np.squeeze(gender_direction)
bias_direction


array([ 0.06651511,  0.15298505, -0.22629546,  0.01367087,  0.10383943,
        0.19781574,  0.14934979,  0.02839268,  0.19078207, -0.06777252,
        0.10532494, -0.11232515,  0.06692035, -0.10428741,  0.06449812,
        0.03163765, -0.20662562, -0.00147265,  0.33190876,  0.14005051,
       -0.00183823,  0.14094647,  0.00308689,  0.16779467,  0.10143815,
        0.18567674, -0.02985709,  0.09640412,  0.06599609, -0.28146555,
       -0.09386818,  0.17567203,  0.11368872,  0.05253632, -0.17035815,
        0.0029247 ,  0.00165801,  0.06263107,  0.1276077 , -0.07693021,
       -0.0652855 , -0.2570974 ,  0.20480825, -0.14751672, -0.0626659 ,
        0.00278824,  0.0045029 , -0.34939256,  0.14731076, -0.10574462])

In [23]:
gender_direction = identify_bias_subspace(
    dict_vec_cleaned, def_set_gender, 1, centralizing=True)



number of words considered: 2
Running PCA with 1 components


In [24]:
gender_direction.ndim

2

In [19]:
random.seed(0)
np.random.seed(0)

X = np.concatenate((masc_vecs, fem_vecs, neut_vecs), axis = 0)
#X = (X - np.mean(X, axis = 0, keepdims = True)) / np.std(X, axis = 0)
y_masc = np.ones(masc_vecs.shape[0], dtype = int)
y_fem = np.zeros(fem_vecs.shape[0], dtype = int)
y_neut = -np.ones(neut_vecs.shape[0], dtype = int)
#y = np.concatenate((masc_scores, fem_scores, neut_scores))#np.concatenate((y_masc, y_fem))
y = np.concatenate((y_masc, y_fem, y_neut))
X_train_dev, X_test, y_train_dev, Y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train, X_dev, Y_train, Y_dev = sklearn.model_selection.train_test_split(X_train_dev, y_train_dev, test_size = 0.3, random_state = 0)
print("Train size: {}; Dev size: {}; Test size: {}".format(X_train.shape[0], X_dev.shape[0], X_test.shape[0]))

Train size: 10788; Dev size: 4624; Test size: 6606


In [20]:
gender_clf = LinearSVC
#gender_clf = SGDClassifier
#gender_clf = LogisticRegression
#gender_clf = LinearDiscriminantAnalysis
#gender_clf = Perceptron

params_svc = {'penalty': 'l2','fit_intercept': False, 'class_weight': None, "dual": False, 'random_state': 0}
params_sgd = {'penalty': 'l2','fit_intercept': False, 'class_weight': None, 'max_iter': 1000, 'random_state': 0}
params = params_svc
#params = {'loss': 'hinge', 'n_jobs': 16, 'penalty': 'l2', 'max_iter': 2500, 'random_state': 0}
#params = {}
n = 35
min_acc = 0
dropout_rate = 0


In [21]:
P, rowspace_projs, Ws = get_debiasing_projection(gender_clf, params, n, 50, min_acc,
                                    X_train, Y_train, X_dev, Y_dev, Y_train_main=None, Y_dev_main=None, 
                                       dropout_rate = 0)

iteration: 34, accuracy: 0.972318339100346: 100%|██████████| 35/35 [00:03<00:00,  9.57it/s]
